**Упражнение 1.**
Перед вами фрагмент кода, содержащего некоторую проблему. Всегда ли counter = 10 после запуска программы?

In [17]:
import threading
import sys

def thread_job():
    global counter
    old_counter = counter
    counter = old_counter + 1
    print('{} '.format(counter), end='')
    sys.stdout.flush()


counter = 0
threads = [threading.Thread(target=thread_job) for _ in range(10)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()
print(counter)

1 2 3 4 5 6 7 8 9 10 10


Для наглядности продемонстрируем "проблему"

In [17]:
import threading
import random
import time
import sys

def thread_job():
    global counter
    old_counter = counter
    time.sleep(random.randint(0, 1))
    counter = old_counter + 1
    print('{} '.format(counter), end='')
    sys.stdout.flush()


counter = 0
threads = [threading.Thread(target=thread_job) for _ in range(10)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()
print(counter)

1 2 3 4 1 1 4 4 5 5 5


**Объясните почему так происходит?**
Несколько потоков могут одновременно выполнять функцию thread_job() и пытаться изменить значение глобальной переменной counter, что может привести к непредсказуемым результатам. Например, если два потока одновременно прочитают значение counter, сохранят его в old_counter, а затем увеличат на 1 и присвоят новое значение counter, то только один из них фактически увеличит counter на 1, а второй поток потеряет свои изменения.
Для избежания race condition в этом коде можно использовать блокировку, чтобы гарантировать, что только один поток может изменять значение counter в данный момент времени.

**Исправьте проблему.**


In [ ]:

import threading
import random
import time
import sys

def thread_job():
    global counter
    lock.acquire()
    old_counter = counter
    time.sleep(random.randint(0, 1))
    counter = old_counter + 1
    print('{} '.format(counter), end='')
    lock.release()
    sys.stdout.flush()

counter = 0
lock = Lock()
threads = [threading.Thread(target=thread_job) for _ in range(10)]


for thread in threads:
    thread.start()
    
    
for thread in threads:
    thread.join()
print(counter)

**Упражнение 2.**
Программист хочет узнать доступность набора ip адресов. Он реализовал программу. Почему она неэффективна? Переделайте с использованием threading. Измерить время с применением потоков и без них. Объяснить результат.

In [16]:
import os, re
import time

received_packages = re.compile(r"(\d) received")

def status(x):
    if x == 0:
        return "no response"
    elif x == 1:
        return "losses"
    elif x == 2:
        return "alive"

time0 = time.time()

for suffix in range(0, 100):
    ip = "192.168.178." + str(suffix)
    ping_out = os.popen("ping -q -c2 " + ip, "r")  # получение вердикта
    #print("... pinging ", ip)
    while True:
        line = ping_out.readline()
        if not line:
            break
        n_received = received_packages.findall(line)
        #if n_received:
    print("Status: ", ip, status(-1))

print(f"time: ", time.time() - time0)

Status:  192.168.178.0 None
Status:  192.168.178.1 None
Status:  192.168.178.2 None
Status:  192.168.178.3 None
Status:  192.168.178.4 None
Status:  192.168.178.5 None
Status:  192.168.178.6 None
Status:  192.168.178.7 None
Status:  192.168.178.8 None
Status:  192.168.178.9 None
Status:  192.168.178.10 None
Status:  192.168.178.11 None
Status:  192.168.178.12 None
Status:  192.168.178.13 None
Status:  192.168.178.14 None
Status:  192.168.178.15 None
Status:  192.168.178.16 None
Status:  192.168.178.17 None
Status:  192.168.178.18 None
Status:  192.168.178.19 None
Status:  192.168.178.20 None
Status:  192.168.178.21 None
Status:  192.168.178.22 None
Status:  192.168.178.23 None
Status:  192.168.178.24 None
Status:  192.168.178.25 None
Status:  192.168.178.26 None
Status:  192.168.178.27 None
Status:  192.168.178.28 None
Status:  192.168.178.29 None
Status:  192.168.178.30 None
Status:  192.168.178.31 None
Status:  192.168.178.32 None
Status:  192.168.178.33 None
Status:  192.168.178.34 

In [ ]:
time:  6256.88433098793

In [ ]:
import os
import re
import threading
import time

received_packages = re.compile(r"(\d) received")

def status(n_received):
    if n_received == 0:
        return "no response"
    elif n_received == 1:
        return "losses"
    else:
        return "alive"

def ping_ip(ip):
    ping_out = os.popen("ping -q -c2 " + ip, "r")
    while True:
        line = ping_out.readline()
        if not line:
            break
        n_received = received_packages.findall(line)
        if n_received:
            print("Status: ", ip, status(int(n_received[0])))

time0 = time.time()

threads = []
for suffix in range(0, 100):
    ip = "192.168.178." + str(suffix)
    print("... pinging ", ip)
    thread = threading.Thread(target=ping_ip, args=(ip,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

print(f"time: {time.time() - time0}")


In [ ]:
time: 12.137522220611572

**Объясните:**
Код, который использует threading, работает быстрее за, потому что он позволяет одновременно проверять несколько IP-адресов, тогда как оригинальный код проверяет каждый IP-адрес последовательно.
Когда оригинальный код проверяет IP-адрес, он ждет ответа на пинг перед переходом к следующему IP-адресу. Это означает, что если один IP-адрес долго не отвечает, весь процесс программы будет задержан. В отличие от этого, код, который использует threading, создает отдельный поток для проверки каждого IP-адреса, так что программа может продолжать проверку других IP-адресов в ожидании ответа от одного IP-адреса.

**Упражнение 3.**
Составить программу, которая считает сумму элементов массива (создать из K значений и заполнить случайным образом) с использованием N потоков. Запустить с разным параметром N (2, 4, 8, 16, 32, 64). Объяснить результат (потребуется измерить время).

In [ ]:

from threading import Thread
import random
import time

K = 1000
N = 4

arr = [random.randint(0, 100) for _ in range(K)]
sum_arr = [0 for _ in range(N)]
total = 0


def sum_array(array):
    global total
    for element in array:
        total += element
    return total


thread = list(range(N))

time_0 = time.time()
for i in range(N):
    thread[i] = Thread(target=sum_array, args=(arr,))
    thread[i].start()

for i in range(N):
    thread[i].join()

print("Sum is %d" % total)
print(time.time() - time_0)


Этот код неэффективен, потому что он использует глобальную переменную "total" для хранения суммы элементов массива. Каждый поток увеличивает эту переменную, что может привести к гонкам данных и непредсказуемым результатам. Кроме того, в каждом потоке выполняется одинаковый код, что приводит к избыточному использованию ресурсов процессора. С увелечением количества потоков время исполнения только увеличивается при N = 4 время выполнения составляет 0.0005109310150146484, при N = 8 время выполнения 0.0009770393371582031, при N = 64 время выполнения равно 0.006939888000488281. Лучше разделить массив на части и передавать каждому потоку свою часть для вычисления суммы. 

**Упражнение 4.** Запустите на исполнение следующий фрагмент кода, замерив время работы. Перепишите с помощью потоков и опять замерьте время. Объясните результат.

In [21]:
import urllib.request
import time


urls = [
    'https://www.yandex.ru', 'https://www.google.com',
    'https://habrahabr.ru', 'https://www.python.org',
    'https://isocpp.org',
]


def read_url(url):
    with urllib.request.urlopen(url) as u:
        return u.read()


start = time.time()
for url in urls:
    read_url(url)
print(time.time() - start)

3.455579996109009


In [ ]:
time = 2.576047897338867

In [ ]:
import urllib.request
import time
import threading


urls = [
    'https://www.yandex.ru', 'https://www.google.com',
    'https://habrahabr.ru', 'https://www.python.org',
    'https://isocpp.org',
]


def read_url(url):
    with urllib.request.urlopen(url) as u:
        return u.read()


start = time.time()
threads = []
for url in urls:
    x = threading.Thread(target = read_url, args =(url,))
    threads.append(x)
    x.start()

for thread in threads:
    thread.join()
    
print(time.time() - start)

In [ ]:
time = 0.009363889694213867

**Упражнение 5.**
Составить программу, которая имеет общие ресурсы для нескольких потоков. Например, есть общая переменная, один поток добавляет 1, второй увеличивает значение в 2 раза. Написать с использованием Lock. Продемонстрировать проблему взаимной блокировки. Исправить её, написав код с использованием RLock блокировки.

In [ ]:
import threading

shared_resource = 0

lock = threading.Lock()


def increment_shared_resource():
    with lock:
        print(f"Thread {threading.current_thread().name} acquired the lock")
        global shared_resource
        shared_resource += 1
        print(f"Thread {threading.current_thread().name} incremented the shared resource to {shared_resource}")
        multiply_shared_resource()


def multiply_shared_resource():
    with lock:
        print(f"Thread {threading.current_thread().name} acquired the lock again")
        global shared_resource
        shared_resource *= 2
        print(f"Thread {threading.current_thread().name} multiplied the shared resource to {shared_resource}")


t1 = threading.Thread(target=increment_shared_resource, name='Thread 1')
t2 = threading.Thread(target=multiply_shared_resource, name='Thread 2')

t1.start()
t2.start()

t1.join()
t2.join()

print(f"The final value of the shared resource is {shared_resource}")


In [ ]:
import threading

shared_resource = 0

lock = threading.RLock()


def increment_shared_resource():
    with lock:
        print(f"Thread {threading.current_thread().name} acquired the lock")
        global shared_resource
        shared_resource += 1
        print(f"Thread {threading.current_thread().name} incremented the shared resource to {shared_resource}")
        multiply_shared_resource()


def multiply_shared_resource():
    with lock:
        print(f"Thread {threading.current_thread().name} acquired the lock again")
        global shared_resource
        shared_resource *= 2
        print(f"Thread {threading.current_thread().name} multiplied the shared resource to {shared_resource}")


t1 = threading.Thread(target=increment_shared_resource, name='Thread 1')
t2 = threading.Thread(target=multiply_shared_resource, name='Thread 2')

t1.start()
t2.start()

t1.join()
t2.join()

print(f"The final value of the shared resource is {shared_resource}")


В этом примере мы создаем общий ресурс (переменную с именем shared_resource) и затем создаем объект RLock с именем lock. Мы определяем две функции, которые обращаются к общему ресурсу (increment_shared_resource и multiply_shared_resource) и оборачиваем каждую функцию в блок with lock:, чтобы гарантировать, что только один поток может обращаться к общему ресурсу в любой момент времени. Мы используем RLock вместо обычного Lock, потому что функция multiply_shared_resource вызывает функцию increment_shared_resource, и мы хотим позволить потоку многократно получать блокировку, не блокируя себя. В первом примере мы использовали обычный Lock и функция multiply_shared_resource заблокировала  себя при попытке повторно получить блокировку, что привело к взаимной блокировке (deadlock).